# Backpropagation Derivations

in this notebook i will explore backpropagation by deriving the gradients used for training nets using backpropagation. Im following along while reading the rumelhart 1986 paper "Learning representations by back-propagating errors" as well as some youtube videos by 3blue1brown and Andrew Ng. 

In [26]:
import sympy as sy
from sympy import Symbol, Function

in a neural network, layers are often refered to as "hidden" if they are not the final layer or the input layer. With this in mind, the cost function for the outcome of the net depends only on the final layer. like in most learning algorithms, the point of the cost function is that we can use it to minimize the difference between the outcome of the model and the truth. if we call the outcome $\hat{y}$ and the truth $y$, then:  $$C_0 = \left( \hat{y} - y \right)^2$$


we know that the outcome depends only on the activation of the final layer. we can call the activation of the final layer $a^{(L)}$, with the superscript $L$ indicating the final layer (we can say the net has $L$ layers). Theres no subscript here indicated which neuron in the layer we are describing the activation of - imagine this net has one neuron per layer for now.

So then the outcome of the net is described by the activation:

$$C_0 = \left( a^{(L)} - y \right)^2$$

we know that in a net, the activation of layer is a function of the activation of the previous layer, multiplied by weights, plus a bias.

$$a^{(L)} = \sigma \left( w^{(L)} a^{(L - 1)} + b^{(L)} \right) $$

we are using sigma here because the activation function can be a function like a sigmoid or relu.

for the calculus later, we are going to give a variable name to the inside of the function:

$$a^{(L)} = \sigma \left( z^{(L)}  \right) $$
where
$$ z^{(L)} = w^{(L)} a^{(L - 1)} + b^{(L)} $$

we have only describes the activation of the final layer in terms of the next layer down - really this is also dependent on the layer before it, and so on until we get to the input vector. 

we can see that the cost function is functionally dependent on the input data (via the activation of layers), the weights, the biases, and the truth. treating the truth and the inputs as static parameters, we are interested in how the weights and biases affect the cost function. 

Firstly, look at just the weights.

So we are interested in the rate of change of the cost function $C_0$ with respect to the weights $w$. Thats the partial derivitive of $C$ with respect to $w$:

$$\frac{\partial C_0}{\partial w^{(L)}}$$

we have already described how the weights are an input at one end of a chain, and the cost function is at the other end of that chain

$$\frac{\partial C_0}{\partial w^{(L)}} = \frac{\partial C_0}{\partial a^{(L)}}\frac{\partial a^{(L)}}{\partial z^{(L)}}\frac{\partial z^{(L)}}{\partial w^{(L)}}$$


we will do the same for the biases, this is going to look almost exactly the same

$$\frac{\partial C_0}{\partial b^{(L)}} = \frac{\partial C_0}{\partial a^{(L)}}\frac{\partial a^{(L)}}{\partial z^{(L)}}\frac{\partial z^{(L)}}{\partial b^{(L)}}$$

In [27]:
# defining symbols for the variable in the above expressions

w = Symbol("w") #weights
b = Symbol("b") #biases

y = Symbol("y") #truth
a_prev = Symbol("a_prev") # activation of previous layer. we are going to treat this as static for now

sigma = Function("sigma")


In [28]:
# we can build the intermediate fuctions z and a (activation of final layer) from these
# type: ignore

z = w * a_prev + b  
z

a_prev*w + b

In [29]:
# type: ignore
a = sigma(z)
a

sigma(a_prev*w + b)

In [30]:
C = (a - y)**2
C

(-y + sigma(a_prev*w + b))**2

$$\frac{\partial C_0}{\partial w^{(L)}} = \frac{\partial C_0}{\partial a^{(L)}}\frac{\partial a^{(L)}}{\partial z^{(L)}}\frac{\partial z^{(L)}}{\partial w^{(L)}}$$

we can substitude the expressions back into each variable to see how we can partially differentiate each one


$$\frac{\partial z^{(L)}}{\partial w^{(L)}} = \frac{\partial w^{(L)} a^{(L - 1)} + b^{(L)}}{\partial w^{(L)}} = a^{(L - 1)}$$


this one is straightforward. 


$$\frac{\partial a^{(L)}}{\partial z^{(L)}} = \frac{\partial \sigma \left( z^{(L)}  \right)}{\partial z^{(L)}}$$


in this term, we have to differentiate the activation function. since im not defining what activation function im using in this nb, ill leave it in symbolic notation

$$\frac{\partial C_0}{\partial a^{(L)}} = \frac{\partial \left( a^{(L)} - y \right)^2}{\partial a^{(L)}}

to evaluate the partial derivitive of the cost function with respect to either wieghts or biases, we would apply the chain rule 

we can use sympy to do these steps for partial derivitives of the cost function by both weights and biases

In [31]:
dC_dw = sy.diff(C, w)
dC_db = sy.diff(C, b)

In [32]:
dC_db

2*(-y + sigma(a_prev*w + b))*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, a_prev*w + b)

In [33]:
dC_dw

2*a_prev*(-y + sigma(a_prev*w + b))*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, a_prev*w + b)

lgtm :)

so far, we have described how backpropagation would work if:

- we only look one layer back from the output layer
- each layer has only one neuron
- we only have one training example

lets try and generalize to nets without this assumptions using intuative explainations

we only look one layer back in the network to the activation of the previous layer, but we know that this layer's activation depends on its weights and biases, and the activation of the layer before it. So we can see that the expressions are identical for this layer, but with different indexes for each layer (i.e. $a^{(L-1)}$, $a^{(L-2)}$, $a^{(L-3)}$). this explains why we call it backpropagation - we keep going backwards through the network, adding in the effects of the weights and balances at each step. Each time we go back a layer and substitute out a_previous in the differential equation, we are going to add two more terms to the chain of partial differentials. 

we only looked at one neuron, and there are no subscripts given to any of the variables. for a more complex net, we can give a subscript to each neuron to keep track of it. each partial derivitive of the cost function (at each index) is then a componant of the overall cost function. A vector of all of these componants describes the gradient of a surface. gradient decent is used to step down this gradient to a minimum point - this is the neural network learning. 

we only have one training example - in the original expressions we give the cost function the subscript 0 to denote that it is specific to a single observation. The actual cost function is going to be an average across all training examples. in practice, we can user stochastic gradient decent to use a subset of observations at each step. 